In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#used to change all the relive dates to datetimes
import datetime

# this in important because the time stamps in the dataset are relitive to when they were scraped
DATE_SCRAPED="03-1-2020"

In [6]:
# read in the data frame that has my scrapped content
df=pd.read_csv('calz_reviews_content.csv')
df_labels=pd.read_csv('calz_labels.csv')


# **EDA**

In [7]:
# inspect head @TODO DROP NAME COL
print(df.shape)
df.head()


(1041, 8)


,web-scraper-order,web-scraper-start-url,name,review,review_content,time_published,owner_response,owner_response_time
0,1583045461-1042,https://www.google.com/maps/place/Cal'z+Pizza+...,Jeanette Zeeb,"<span jstcache=""293"" jsinstance=""0"" class=""sec...",It's ok,a year ago,Response from the owner,a year ago
1,1583045296-198,https://www.google.com/maps/place/Cal'z+Pizza/...,Deanna Frantzen,"<span jstcache=""293"" jsinstance=""0"" class=""sec...",Great food and amazing staff,a year ago,NaN,NaN
2,1583045434-829,https://www.google.com/maps/place/Cal'z+Pizza/...,Monique Shontae,"<span jstcache=""293"" jsinstance=""0"" class=""sec...",Order a tuna cheese sub Friday night.. It had ...,a year ago,NaN,NaN
3,1583045434-869,https://www.google.com/maps/place/Cal'z+Pizza/...,Annie Cheeks,"<span jstcache=""293"" jsinstance=""0"" class=""sec...","Great service with a smile, the young man was ...",2 years ago,NaN,NaN
4,1583045434-899,https://www.google.com/maps/place/Cal'z+Pizza/...,mattthew kinnard,"<span jstcache=""293"" jsinstance=""0"" class=""sec...",Good pizza,2 years ago,NaN,NaN


In [8]:
print(df_labels.shape)
df_labels.head(20)

(9, 6)


,web-scraper-order,web-scraper-start-url,name,address,rating,num_to_rate
0,1583044594-2,https://www.google.com/maps/place/Cal'z+Pizza+...,Cal'z Pizza,3678 Sewells Point Rd,4.0,-322
1,1583044594-5,https://www.google.com/maps/place/Cal'z+Pizza/...,Cal'z Pizza,3324 Holland Road,4.0,-192
2,1583044594-7,https://www.google.com/maps/place/Cal'z+Pizza/...,Cal'z Pizza,1009 Laskin Rd,4.2,-251
3,1583044594-4,https://www.google.com/maps/place/Cal'z+Pizza/...,Cal'z Pizza,5280 Princess Anne Rd,4.4,-165
4,1583044594-9,https://www.google.com/maps/place/Cal'z+Pizza/...,Cal'z Pizza,624 Independence Blvd,3.9,-133
5,1583044594-3,"https://www.google.com/maps/place/Cal'z+Pizza,...",Cal'z Pizza,738 E Bayview Blvd,4.1,-160
6,1583044594-1,https://www.google.com/maps/place/Cal'z+Pizza/...,Cal'z Pizza - East Little Creek,3824 E Little Creek Rd,4.2,-172
7,1583044594-8,https://www.google.com/maps/place/Cal'z+Pizza/...,Cal'z Pizza,1405 Harpers Rd,4.4,-82
8,1583044594-6,https://www.google.com/maps/place/Cal'z+Pizza/...,"Cal'z Pizza, Subs, & Wings",8245 Hampton Blvd,3.8,-86


In [10]:
# define a function that will take the cell contents of num_to_rate and remove the
# paretheses
def strip_n2r(x):
    return (-1)*x
test=df_labels['num_to_rate'].apply(strip_n2r)
test.head()

0    322
1    192
2    251
3    165
4    133
Name: num_to_rate, dtype: int64

In [11]:
# make changes to df
df_labels['num_to_rate']=df_labels['num_to_rate'].apply(strip_n2r)

#Data Cleaning/feature engineering

###**geo-codes engineering**

In [12]:
# im going to have to extract the geo code from the following links and compare them to my geo codes from
# my labels df to map the reviews to a particular store
print("labels df:")
print(df_labels['web-scraper-start-url'].iloc[0])
print("\nreviews df:")
df['web-scraper-start-url'].iloc[0]

labels df:
https://www.google.com/maps/place/Cal'z+Pizza+-+East+Little+Creek/@36.9181643,-76.2003157,17z/data=!3m1!4b1!4m10!1m2!2m1!1scalz+pizza!3m6!1s0x89ba914bb73fa53b:0x1301c924d4200997!8m2!3d36.91816!4d-76.198127!9m1!1b1

reviews df:


"https://www.google.com/maps/place/Cal'z+Pizza+-+East+Little+Creek/@36.9181643,-76.2003157,17z/data=!3m1!4b1!4m10!1m2!2m1!1scalz+pizza!3m6!1s0x89ba914bb73fa53b:0x1301c924d4200997!8m2!3d36.91816!4d-76.198127!9m1!1b1"

In [13]:
# define a function to strip the url strings to reveal geo tags
def parse_geo(x):
  return x.split('@')[1].split(',')[:-1]
# test it out before i make any changes
# note i probably need to only test on one of the df's because the
# start url is in the same format in both df's
test=df['web-scraper-start-url'].apply(parse_geo)
test.head()

0    [36.9181643, -76.2003157]
1    [36.8565404, -75.9934831]
2    [36.8892982, -76.2401211]
3    [36.8892982, -76.2401211]
4    [36.8892982, -76.2401211]
Name: web-scraper-start-url, dtype: object

In [14]:
# make changes
df['geo']=df['web-scraper-start-url'].apply(parse_geo)
df_labels['geo']=df_labels['web-scraper-start-url'].apply(parse_geo)

###**rating valuation engineering**

In [15]:
# on the reviews it had images of lit stars triggered by js
# to work around this i took the html source because i knew it would
# show which stars are supossed to be triggered, but as a drawback
# no i have to clean HTML instead of pretty output
df['review'].head()

0    <span jstcache="293" jsinstance="0" class="sec...
1    <span jstcache="293" jsinstance="0" class="sec...
2    <span jstcache="293" jsinstance="0" class="sec...
3    <span jstcache="293" jsinstance="0" class="sec...
4    <span jstcache="293" jsinstance="0" class="sec...
Name: review, dtype: object

In [16]:
# make a funtion to apply to col
def clean_rating(x):
  # make a list of strings to work with
  out=x.split()
  # sort list so that duplicates group togeather
  out.sort()
  # the first element is just '<span>' and the following 20 elements are redundant,
  # the last 5 elements are placment flags
  # remove all of them and leave me with just the number of active stars
  # then get a count so i can have a pretty int
  return len(out[21:len(out)-5])

# test to make sure that it works
test=df['review'].apply(clean_rating)
test.head()

0    3
1    5
2    1
3    5
4    4
Name: review, dtype: int64

In [17]:
# make changes to df
df['review']=df['review'].apply(clean_rating)

### **datetime engeenering for graphing**

In [18]:
# humanfied dates are great to read, not so great to graph, i need to make theese
# back into dt format,(why i have a glob at the top with the date of scrape)
df['time_published'].value_counts()

a year ago       328
2 years ago      173
3 years ago       99
11 months ago     45
7 months ago      43
10 months ago     36
a month ago       36
8 months ago      36
6 months ago      35
3 months ago      29
5 months ago      28
4 months ago      26
9 months ago      25
4 years ago       21
5 years ago       15
2 months ago      13
2 weeks ago       10
a week ago         7
6 years ago        7
3 weeks ago        7
4 weeks ago        6
7 years ago        6
2 days ago         3
9 years ago        2
8 years ago        1
a day ago          1
3 days ago         1
6 days ago         1
4 days ago         1
Name: time_published, dtype: int64

In [19]:
# create a map for all units to hours
time_map={'second':0.000278,'minute':0.0167,'hour':1,'day':24,'week':168,'month':530.5,'year':8766}
# make a function that takes a relitive time 'a day ago' and transfers that to a datetime
def human_to_dt(x):
  # the number that i will end up subtracting from scrape date
  diff=0
  # make a datetime object to hold the date that the dataset was scrapped
  dt=datetime.datetime.strptime(DATE_SCRAPED, '%m-%d-%Y')
  # strip the string down to two values a quantifier and a unit
  q,u=x[:-3].strip().split(' ')
  # remove trailing s on unit it is not needed
  if u[len(u)-1]=="s":
    u=u[:len(u)-1]
  # check if there is just one unit if so then set diff=1
  if q == "a":
    diff=time_map[u]
  # if the number is not one then multiply the q by the map_key entry for u
  else:
    #safe cast
    try:
      diff=time_map[u]*int(q)
    except:
      print("ERROR")
      return 0
  # convert dt to utc timestamp
  dt=dt-datetime.timedelta(hours=diff)
  timestamp = dt.replace(tzinfo=datetime.timezone.utc).timestamp()
  return timestamp

In [20]:
# making sure that it works how i want it to
testvalue=df['time_published'].iloc[0]
print(human_to_dt(testvalue))
del testvalue

1551463200.0


In [21]:
# apply changes to df
df['time_published']=df['time_published'].apply(human_to_dt)

## more data cleaning to drop columns that i dont need anymore

In [22]:
df.head()

,web-scraper-order,web-scraper-start-url,name,review,review_content,time_published,owner_response,owner_response_time,geo
0,1583045461-1042,https://www.google.com/maps/place/Cal'z+Pizza+...,Jeanette Zeeb,3,It's ok,1.551463e+09,Response from the owner,a year ago,"[36.9181643, -76.2003157]"
1,1583045296-198,https://www.google.com/maps/place/Cal'z+Pizza/...,Deanna Frantzen,5,Great food and amazing staff,1.551463e+09,NaN,NaN,"[36.8565404, -75.9934831]"
2,1583045434-829,https://www.google.com/maps/place/Cal'z+Pizza/...,Monique Shontae,1,Order a tuna cheese sub Friday night.. It had ...,1.551463e+09,NaN,NaN,"[36.8892982, -76.2401211]"
3,1583045434-869,https://www.google.com/maps/place/Cal'z+Pizza/...,Annie Cheeks,5,"Great service with a smile, the young man was ...",1.519906e+09,NaN,NaN,"[36.8892982, -76.2401211]"
4,1583045434-899,https://www.google.com/maps/place/Cal'z+Pizza/...,mattthew kinnard,4,Good pizza,1.519906e+09,NaN,NaN,"[36.8892982, -76.2401211]"


In [23]:
df_labels.head()

,web-scraper-order,web-scraper-start-url,name,address,rating,num_to_rate,geo
0,1583044594-2,https://www.google.com/maps/place/Cal'z+Pizza+...,Cal'z Pizza,3678 Sewells Point Rd,4.0,322,"[36.9181643, -76.2003157]"
1,1583044594-5,https://www.google.com/maps/place/Cal'z+Pizza/...,Cal'z Pizza,3324 Holland Road,4.0,192,"[36.8892982, -76.2401211]"
2,1583044594-7,https://www.google.com/maps/place/Cal'z+Pizza/...,Cal'z Pizza,1009 Laskin Rd,4.2,251,"[36.9340261, -76.2473017]"
3,1583044594-4,https://www.google.com/maps/place/Cal'z+Pizza/...,Cal'z Pizza,5280 Princess Anne Rd,4.4,165,"[36.8288303, -76.1697097]"
4,1583044594-9,https://www.google.com/maps/place/Cal'z+Pizza/...,Cal'z Pizza,624 Independence Blvd,3.9,133,"[36.8024512, -76.0894311]"


In [24]:
# going to drop internal scraper columns that dont add context for the information
# that is provided by them
df=df.drop(['web-scraper-order','web-scraper-start-url'],axis=1)
df_labels=df_labels.drop(['web-scraper-order','web-scraper-start-url'],axis=1)

##merging df's based on geo codes

In [26]:
# but first i have to cast and round existing geo codes in both df's because
# google does such a great job at consitency

# define a function that reurns a tuple of rounded floates from the geo tags
# going with 3 signifigant figures since that equates to 110m and none of the 
# stores are that close
def clean_geos(x):
  out=[round(float(x[0]),3),round(float(x[1]),3)]
  return out
  
# self explainitory
def sep_lat(x):
  return x[0]
def sep_long(x):
  return x[1]
test=df['geo'].iloc[0]
test_labels=df_labels['geo'].iloc[0]
print(clean_geos(test))
print(clean_geos(test_labels))

[36.918, -76.2]
[36.918, -76.2]


In [27]:
# make changes to df's
df['geo']=df['geo'].apply(clean_geos)
df_labels['geo']=df_labels['geo'].apply(clean_geos)

In [28]:
# add latitude and longitude columns
df['latitude']=df['geo'].apply(sep_lat)
df_labels['latitude']=df_labels['geo'].apply(sep_lat)
df['longitude']=df['geo'].apply(sep_long)
df_labels['longitude']=df_labels['geo'].apply(sep_long)

In [29]:
df_labels.head()

,name,address,rating,num_to_rate,geo,latitude,longitude
0,Cal'z Pizza,3678 Sewells Point Rd,4.0,322,"[36.918, -76.2]",36.918,-76.200
1,Cal'z Pizza,3324 Holland Road,4.0,192,"[36.889, -76.24]",36.889,-76.240
2,Cal'z Pizza,1009 Laskin Rd,4.2,251,"[36.934, -76.247]",36.934,-76.247
3,Cal'z Pizza,5280 Princess Anne Rd,4.4,165,"[36.829, -76.17]",36.829,-76.170
4,Cal'z Pizza,624 Independence Blvd,3.9,133,"[36.802, -76.089]",36.802,-76.089


In [30]:
df.head()

,name,review,review_content,time_published,owner_response,owner_response_time,geo,latitude,longitude
0,Jeanette Zeeb,3,It's ok,1.551463e+09,Response from the owner,a year ago,"[36.918, -76.2]",36.918,-76.200
1,Deanna Frantzen,5,Great food and amazing staff,1.551463e+09,NaN,NaN,"[36.857, -75.993]",36.857,-75.993
2,Monique Shontae,1,Order a tuna cheese sub Friday night.. It had ...,1.551463e+09,NaN,NaN,"[36.889, -76.24]",36.889,-76.240
3,Annie Cheeks,5,"Great service with a smile, the young man was ...",1.519906e+09,NaN,NaN,"[36.889, -76.24]",36.889,-76.240
4,mattthew kinnard,4,Good pizza,1.519906e+09,NaN,NaN,"[36.889, -76.24]",36.889,-76.240


In [31]:
# drop some more columns that are a pain in my ass
df=df.drop('geo',axis=1)
df_labels=df_labels.drop('geo',axis=1)

# merge data frames which will map the store the customer
test=df.merge(df_labels,on=['latitude','longitude'])

In [34]:
# grab a random sample to see what the dataframe looks like
test.sample(40)

,name_x,review,review_content,time_published,owner_response,owner_response_time,latitude,longitude,name_y,address,rating,num_to_rate
607,Beautiful Because,3,Cal'z has definitely went down hill. Im not sa...,1.551463e+09,NaN,NaN,36.802,-76.089,Cal'z Pizza,624 Independence Blvd,3.9,133
281,Ali Wolf975,5,This was the first time I ordered from them! I...,1.567742e+09,NaN,NaN,36.889,-76.240,Cal'z Pizza,3324 Holland Road,4.0,192
166,Stephanie Lambert,5,The best location ever!,1.551463e+09,NaN,NaN,36.857,-75.993,Cal'z Pizza - East Little Creek,3824 E Little Creek Rd,4.2,172
810,Amber Rogers,5,I absolutely love their food,1.551463e+09,NaN,NaN,36.855,-76.137,"Cal'z Pizza, Subs, & Wings",8245 Hampton Blvd,3.8,86
685,Diego A. Matom,3,NaN,1.551463e+09,NaN,NaN,36.802,-76.089,Cal'z Pizza,624 Independence Blvd,3.9,133
0,Jeanette Zeeb,3,It's ok,1.551463e+09,Response from the owner,a year ago,36.918,-76.200,Cal'z Pizza,3678 Sewells Point Rd,4.0,322
912,allen bailey,5,Great,1.551463e+09,NaN,NaN,36.829,-76.170,Cal'z Pizza,5280 Princess Anne Rd,4.4,165
430,Charles Cook,2,Wings small. fries mushy and damp. A small po...,1.579201e+09,NaN,NaN,36.889,-76.240,Cal'z Pizza,3324 Holland Road,4.0,192
101,David Francis,4,Good food,1.488348e+09,NaN,NaN,36.918,-76.200,Cal'z Pizza,3678 Sewells Point Rd,4.0,322
599,Marc Amidon,5,Great food 👍👍👍,1.562013e+09,NaN,NaN,36.802,-76.089,Cal'z Pizza,624 Independence Blvd,3.9,133


##**I think thats gonna be good for this dataset, lets do some house keeping and export it to a file to use**

In [35]:
print(test.columns.to_list())

['name_x', 'review', 'review_content', 'time_published', 'owner_response', 'owner_response_time', 'latitude', 'longitude', 'name_y', 'address', 'rating', 'num_to_rate']


In [36]:
# rename my columns to something easier then x_name y_name ect
df=test
perfered_names=['customer_name', 'review_rating', 'review_content','time', 'owner_response','owner_response_time','latitude',
 'longitude', 'store_name', 'store_address','store_rating','num_to_rate']
df.columns=perfered_names


In [37]:
df.columns.to_list()

['customer_name',
 'review_rating',
 'review_content',
 'time',
 'owner_response',
 'owner_response_time',
 'latitude',
 'longitude',
 'store_name',
 'store_address',
 'store_rating',
 'num_to_rate']

In [39]:
perf_order=['time',
            'customer_name',
            'review_content',
            'review_rating',
            'store_name',
            'store_rating',
            'store_address',
            'num_to_rate',
            'owner_response',
            'owner_response_time',
            'latitude',
            'longitude']
df=df[perf_order]

In [40]:
df.head()

,time,customer_name,review_content,review_rating,store_name,store_rating,store_address,num_to_rate,owner_response,owner_response_time,latitude,longitude
0,1.551463e+09,Jeanette Zeeb,It's ok,3,Cal'z Pizza,4.0,3678 Sewells Point Rd,322,Response from the owner,a year ago,36.918,-76.2
1,1.519906e+09,Daniel Mcquitty,Food was great and fast service.,5,Cal'z Pizza,4.0,3678 Sewells Point Rd,322,NaN,NaN,36.918,-76.2
2,1.577291e+09,Kuki Hell,DELICIOUS. So far had the calzown and it was b...,5,Cal'z Pizza,4.0,3678 Sewells Point Rd,322,NaN,NaN,36.918,-76.2
3,1.581111e+09,Thomas Hall,Small wings a bit overpriced,3,Cal'z Pizza,4.0,3678 Sewells Point Rd,322,NaN,NaN,36.918,-76.2
4,1.565833e+09,Mister Henderson,fast service good food,4,Cal'z Pizza,4.0,3678 Sewells Point Rd,322,NaN,NaN,36.918,-76.2


In [41]:
df.to_csv('calz_processed.csv')